In [17]:
 #cadena=input()
cadena="quisiera saber las noticias de hoy"

In [18]:
def correcciones(cadena):
   
    import spacy
    nlp=spacy.load('es_core_news_lg')
    from nltk.tokenize import word_tokenize
    import re
    from spacy.matcher import Matcher
    palabra_frecuencia = {}
    global pregunta_listata
    with open(r"Palabra_Frecuencia.txt",encoding='utf-8') as f:
        for linea in f:
            (key, val) = linea.split('#')
            palabra_frecuencia[key] = int(val)

    def frecuencia(p):
        try:
            return palabra_frecuencia[p]
        except:
            return 0



    def remove_repeated_characters(tokens,s=100): 
       repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)') 
       match_substitution = r'\1\2\3'

       def replace(old_word):
          if frecuencia(old_word)>s:
             return old_word
          new_word = repeat_pattern.sub(match_substitution, old_word)
          return replace(new_word) if new_word != old_word else new_word

       correct_tokens = [replace(word) for word in tokens]
       return correct_tokens


    #convierto en cadena la lista
    cadena="".join(cadena)
    doc = nlp(cadena)
    tokens_str=[str(x) for x in doc]
    cadena=remove_repeated_characters(tokens_str)


    def words(text): return re.findall(r'\w+', text.lower())

    WORDS = palabra_frecuencia

    def corregir(oracion):
        oracion_corregida=''
        for palabra in re.findall(r'\w+',oracion):
            oracion_corregida += correction(palabra)[0] + ' '
        return oracion_corregida

    def correction(word): 
        "Most probable spelling correction for word."
        return [max(candidates(word), key=frecuencia)]

    def candidates(word): 
        "Generate possible spelling corrections for word."
        return (known([word]) or 
                known(edits1(word)) or 
                #known(edits2(word)) or 
                [word])

    def known(words): 
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in WORDS)

    def edits1(word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnñopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]


        deletes    = [L + R[1:]               for L, R in splits if R]


        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]


        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]

        inserts    = [L + c + R               for L, R in splits for c in letters]

        return set(deletes + transposes + replaces + inserts)

    def edits2(word): 
        "All edits that are two edits away from `word`."
        return (e2 for e1 in edits1(word) for e2 in edits1(e1))


    corregida=corregir(str(cadena))

     #===ELIMINAR STOP WORDS===

    def stopWords(corpus):
        global oraciones_filtradas
        f = open('stopwords.txt', 'r')
        stopwords = f.read().split('\n')
        f.close()
        " ".join(stopwords)
        oraciones=pd.Series(corpus)

        oraciones_filtradas = [" ".join([
                                  palabra for palabra in oracion.split()
                                  if palabra not in stopwords])
                                  for oracion in oraciones]
    stopWords(corregida)

    #pregunta_lista = " ".join(oraciones_filtradas)

    #===STEMMATIZAR===
    from nltk.stem import SnowballStemmer

    spanish_stemmer = SnowballStemmer('spanish')

    oraciones_filtradas_lematizadas=[]
    for oracion in oraciones_filtradas:
        oracion_lematizada = []
        for palabra in oracion.split():
            doc=nlp(palabra)
            oracion_lematizada.append(doc[0].lemma_)
        oraciones_filtradas_lematizadas.append(" ".join(oracion_lematizada))

    global pregunta_lista
    oraciones_filtradas_lematizadas_stemm=[]

    for oracion in oraciones_filtradas_lematizadas:
        oracion_stemm = []
        for palabra in oracion.split():
            oracion_stemm.append(spanish_stemmer.stem(palabra))
        oraciones_filtradas_lematizadas_stemm.append(" ".join(oracion_stemm))

    pregunta_lista=oraciones_filtradas_lematizadas_stemm
correcciones(cadena)

In [19]:
pregunta_lista

['quer sab notici hoy']

In [20]:
def prediccion(corpus):
    import pickle
    from sklearn.feature_extraction.text import CountVectorizer
    global result
    global probabilidad
    
    loaded_vec = CountVectorizer(vocabulary=pickle.load(open("prediccion/estructure.pkl", "rb"))) 
    new_input=loaded_vec.transform(corpus)
    new_input=pd.DataFrame(new_input.toarray()) 


    clf_input=pickle.load(open("prediccion/modelo.sav", "rb"))
    probabilidad=sorted(list(clf_input.predict_proba(new_input)[0]))[-1]
    predict_input=clf_input.predict(new_input)

 
    for index,value in enumerate(predict_input):
        result=[]
        if float(probabilidad) >= 0.4:
            result.append(predict_input[index])

    
    result=np.unique(result)
    result=result.tolist() 
prediccion(pregunta_lista) 

C:\Users\lobo\anaconda3\envs\IA\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\lobo\anaconda3\envs\IA\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [21]:
result

['noticias']

In [22]:
 probabilidad

0.9245996989680577